### **Модуль детекції вирв та ушкоджень земельних ділянок засобами OpenCV**

Сучасні технології комп'ютерного зору відкривають нові можливості для автоматизованої детекції кратерів та ушкоджень земельних ділянок. Розробка такої системи є актуальною задачею, особливо в контексті моніторингу територій, постраждалих від військових дій, або для аналізу природних катастроф. Запропонована система детекції поєднує різні методи обробки зображень, від базової підготовки даних до складних алгоритмів розпізнавання образів.

#### Функціонально-логічна структура модуля 

##### 1. Блок підготовки зображень

Якість вхідних даних визначає успішність всього процесу детекції, тому етап підготовки зображення є фундаментальним для досягнення точних результатів. Основним інструментом для зменшення шуму при збереженні важливих деталей краю служить двосторонній фільтр cv2.bilateralFilter(). Цей метод особливо ефективний для супутникових знімків, де необхідно зберегти чіткість меж кратерів при одночасному видаленні шуму від атмосферних перешкод або недосконалості сенсорів.

Конвертація кольорових зображень у відтінки сірого за допомогою cv2.cvtColor() спрощує подальшу обробку та знижує обчислювальну складність алгоритмів. Покращення контрастності через cv2.filter2D() дозволяє виділити слабко помітні деталі рельєфу, що критично важливо для виявлення дрібних або частково замаскованих пошкоджень.

Альтернативні підходи до зменшення шуму включають гауссівське розмиття cv2.GaussianBlur() для загального згладжування та медіанний фільтр cv2.medianBlur() для ефективного видалення імпульсного шуму. Морфологічні операції можуть доповнювати основні методи фільтрації, особливо для усунення дрібних артефактів. При конвертації у відтінки сірого weighted average метод дозволяє налаштувати вагові коефіцієнти для різних кольорових каналів, а luminance-based conversion враховує особливості людського сприйняття яскравості.

Для покращення контрастності гістограмна еквалізація cv2.equalizeHist() забезпечує рівномірний розподіл інтенсивності, контрастно-обмежена адаптивна гістограмна еквалізація cv2.createCLAHE() запобігає надмірному підсиленню шуму, а гамма-корекція дозволяє точно налаштувати яскравість у різних діапазонах інтенсивності.

##### 2. Блок детекції контурів 

Виявлення контурів об'єктів становить основу для подальшого розпізнавання кратерів, оскільки більшість пошкоджень земної поверхні характеризуються чіткими межами між пошкодженою та непошкодженою територією. Алгоритм Canny, реалізований через cv2.Canny(), забезпечує оптимальний баланс між точністю детекції та швидкістю обробки. Цей метод використовує двопорогову схему для виділення чітких та розмитих контурів, що дозволяє виявляти як чіткі контури великих кратерів, так і менш виражені межі дрібних ушкоджень.

Градієнтний оператор Собеля cv2.Sobel() пропонує альтернативний підхід, особливо ефективний для виявлення країв у певному напрямку. Це корисно при аналізі лінійних пошкоджень або кратерів із характерною орієнтацією. Лапласіан cv2.Laplacian(), як оператор другої похідної, чутливий до швидких змін інтенсивності та може виявляти тонкі деталі контурів, хоча він більш схильний до шуму. Оператор Шарра cv2.Scharr() представляє вдосконалену версію Собеля з кращою ротаційною симетрією, що робить його придатним для детекції країв різної орієнтації.

Вибір методу детекції країв залежить від специфіки знімків та типу пошкоджень, які необхідно виявити. Для складних сценаріїв можливе комбінування кількох підходів для досягнення максимальної повноти детекції.

##### 2.1. Постобробка результатів детекції із використанням морфологічних операцій

Після детекції країв часто виникає необхідність у додатковій обробці для покращення якості контурів та усунення розривів у межах об'єктів. Операція розширення cv2.dilate() відіграє ключову роль у з'єднанні переривчастих країв кратерів, які могли бути пошкоджені шумом або недостатньою роздільною здатністю зображення. Цей метод особливо корисний при роботі із супутниковими знімками низької якості або при наявності часткових затінень.

Операція звуження cv2.erode() служить для видалення дрібних артефактів та тонких з'єднань між об'єктами, що помилково можуть бути розпізнані як частини кратерів. Комплексні морфологічні операції cv2.morphologyEx() надають ширші можливості для обробки: операція opening (звуження з подальшим розширенням) ефективно усуває шум при збереженні основних контурів, closing (розширення з подальшим звуженням) заповнює дірки та розриви в об'єктах, а gradient операція виділяє межі об'єктів.

Правильний вибір структуруючого елементу та кількості ітерацій морфологічних операцій критично впливає на якість результату. Для кругових об'єктів доцільно використовувати еліптичні або кругові ядра, тоді як для лінійних пошкоджень краще підходять прямокутні структуруючі елементи.

##### 2.2. Детекція кратерів із використанням фільтрів для детекції кіл

Більшість кратерів та вибухових пошкоджень мають приблизно круглу або еліптичну форму, що робить детекцію кіл центральним елементом системи розпізнавання. Трансформація Хафа для кіл cv2.HoughCircles() є стандартним та найпоширенішим методом для виявлення кругових об'єктів на зображеннях. Алгоритм працює у просторі параметрів, накопичуючи голоси за потенційні центри та радіуси кіл, що дозволяє виявляти навіть частково видимі або спотворені круглі об'єкти.

Template matching через cv2.matchTemplate() пропонує альтернативний підхід, особливо ефективний коли відома приблизна форма та розмір кратерів. Цей метод дозволяє шукати специфічні патерни пошкоджень та може бути налаштований для розпізнавання кратерів із характерними особливостями, такими як підвищені краї або центральні заглиблення.

Контурний аналіз через cv2.findContours() у поєднанні з апроксимацією еліпсів cv2.fitEllipse() надає гнучкіший підхід до детекції об'єктів неправильної форми. Цей метод дозволяє виявляти не тільки ідеально круглі кратери, але й деформовані або частково зруйновані об'єкти.

Сучасні підходи на основі глибокого навчання, такі як YOLO та Faster R-CNN, відкривають нові можливості для високоточної детекції складних об'єктів. Ці методи можуть навчатися на великих датасетах зображень кратерів та автоматично виявляти складні патерни, недоступні для традиційних алгоритмів комп'ютерного зору.

##### 3. Географічні трансформації

Перетворення координат пікселів у реальні географічні координати є критично важливим для практичного застосування системи детекції. Кожен виявлений кратер повинен бути прив'язаний до конкретної локації на земній поверхні з високою точністю. Процес конвертації враховує рівень масштабування карти та роздільну здатність зображення, оскільки ці параметри безпосередньо впливають на відповідність між пікселями та реальними відстанями.

Web Mercator проекція широко використовується у сучасних картографічних сервісах та забезпечує стандартизований спосіб представлення географічних даних. Використання цієї проекції дозволяє легко інтегрувати результати детекції з існуючими геоінформаційними системами та забезпечити сумісність із різними картографічними платформами.

Точність географічної прив'язки особливо важлива для координації аварійно-рятувальних робіт або планування відновлювальних заходів. Система повинна враховувати можливі спотворення проекції та забезпечувати коректну калібровку для різних регіонів та масштабів зображень.

##### 4. Оптимізація параметрів системи детекції ушкоджень

Ефективність системи детекції суттєво залежить від правильного налаштування численних параметрів алгоритмів, що робить автоматизовану оптимізацію критично важливою для досягнення максимальної точності. Інтеграція з бібліотекою Optuna дозволяє систематично досліджувати простір параметрів та знаходити оптимальні налаштування для конкретних типів зображень та умов роботи.

Пороги детекції країв у алгоритмі Canny (T1 та T2) визначають чутливість до слабких та сильних країв відповідно. Їх оптимальні значення залежать від контрастності зображень та рівня шуму. Параметри трансформації Хафа для кіл включають роздільну здатність акумулятора dp, мінімальну відстань між центрами кіл min_dist, пороги для детекції країв param1 та центрів кіл param2, а також діапазон радіусів для пошуку.

Налаштування двостороннього фільтра впливає на баланс між збереженням деталей та зменшенням шуму, тоді як параметри морфологічних операцій визначають ступінь модифікації контурів. Оптимізація всіх цих параметрів одночасно є складною багатовимірною задачею, яка може бути ефективно вирішена за допомогою сучасних методів автоматичного налаштування гіперпараметрів.

Застосування Optuna дозволяє не тільки знайти оптимальні налаштування, але й дослідити взаємозв'язки між різними параметрами та їх вплив на якість детекції. Це забезпечує створення робастної системи, здатної адаптуватися до різних умов та типів вхідних даних.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import glob
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass
from pydantic import BaseModel
import optuna
import pandas as pd
import pyproj
from geopy import distance
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
@dataclass
class ImageMetadata:
    """Metadata for satellite images"""
    coordinates: Tuple[float, float]  # (lat, lon)
    resolution: float  # meters per pixel
    zoom_level: int
    image_path: str
    image_size: Tuple[int, int]  # (width, height)

@dataclass
class CraterDetection:
    """Detected crater information"""
    pixel_coords: Tuple[int, int]  # (x, y) in pixels
    radius_pixels: int
    geo_coords: Tuple[float, float]  # (lat, lon)
    radius_meters: float
    confidence: float

class Metrics(BaseModel):
    """Evaluation metrics for crater detection"""
    TP: int
    FP: int
    FN: int
    precision: float
    recall: float
    f1_score: float

In [ ]:
class SatelliteImageProcessor:
    """
    Main class for processing satellite images and detecting craters
    
    Теоретичний опис блоків:
    
    1. Підготовка зображення:
       - Конвертація у відтінки сірого: cv2.cvtColor() - стандартний підхід
       - Альтернативи: weighted grayscale, luminance-based conversion
       - Покращення контрасту: filter2D з кастомним ядром
       - Альтернативи: histogram equalization (equalizeHist), CLAHE, gamma correction
       - Фільтрація шуму: bilateralFilter - зберігає краї
       - Альтернативи: GaussianBlur, medianBlur, morphological operations
    
    2. Детекція країв:
       - Canny Edge Detection: оптимальний баланс точності та швидкості
       - Альтернативи: Sobel, Laplacian, Scharr operators
       - Морфологічні операції: dilate для з'єднання переривчастих країв
       - Альтернативи: opening, closing, gradient operations
    
    3. Детекція кіл:
       - HoughCircles: стандарт для детекції круглих об'єктів
       - Альтернативи: template matching, contour-based detection, 
         machine learning approaches (YOLO, R-CNN)
    
    4. Координатні трансформації:
       - Pixel to geographic conversion з урахуванням zoom level та resolution
       - Використання проекцій (Web Mercator для більшості карт)
    """
    
    def __init__(self, input_folder: str, output_folder: str = "output"):
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.image_metadata = {}
        self.detection_results = {}
        self.optimized_params = None
        
        # Default detection parameters
        self.default_params = {
            'canny_T1': 91,
            'canny_T2': 31,
            'dp': 1.20,
            'min_dist': 50,
            'param1': 50,
            'param2': 15,
            'min_radius': 5,
            'max_radius': 50,
            'bilateral_d': 9,
            'bilateral_sigma_color': 75,
            'bilateral_sigma_space': 75,
            'dilate_iterations': 3
        }
        
        # Morphological kernel for dilation
        self.dilation_kernel = np.ones((3,3), np.uint8)
        
        # Create output folder
        os.makedirs(output_folder, exist_ok=True)
        
        # Load image metadata
        self._load_image_metadata()
    
    def _load_image_metadata(self):
        """Load metadata for satellite images from SatelliteImageRetriever"""
        metadata_file = os.path.join(self.input_folder, "metadata.json")
        
        if os.path.exists(metadata_file):
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
                
            for img_data in metadata:
                img_path = img_data['path']
                base_name = Path(img_path).stem
                
                # Load image to get size
                img = cv2.imread(os.path.join(self.input_folder, img_path))
                if img is not None:
                    height, width = img.shape[:2]
                    
                    self.image_metadata[base_name] = ImageMetadata(
                        coordinates=(img_data['lat'], img_data['lon']),
                        resolution=img_data['resolution'],
                        zoom_level=img_data['zoom'],
                        image_path=img_path,
                        image_size=(width, height)
                    )
        else:
            logger.warning(f"Metadata file not found: {metadata_file}")
            # Fallback: scan folder and create dummy metadata
            self._create_dummy_metadata()
    
    def _create_dummy_metadata(self):
        """Create dummy metadata when metadata.json is not available"""
        image_files = glob.glob(os.path.join(self.input_folder, "*.jpg")) + \
                     glob.glob(os.path.join(self.input_folder, "*.png"))
        
        for img_path in image_files:
            base_name = Path(img_path).stem
            img = cv2.imread(img_path)
            if img is not None:
                height, width = img.shape[:2]
                
                self.image_metadata[base_name] = ImageMetadata(
                    coordinates=(0.0, 0.0),  # Default coordinates
                    resolution=1.0,  # Default resolution
                    zoom_level=15,   # Default zoom
                    image_path=os.path.basename(img_path),
                    image_size=(width, height)
                )
    
    def contrast_enhancement(self, img: np.ndarray, kernel: Optional[np.ndarray] = None) -> np.ndarray:
        """
        Enhance image contrast using convolution
        
        Альтернативи:
        - CLAHE: cv2.createCLAHE()
        - Histogram equalization: cv2.equalizeHist()
        - Gamma correction: np.power(img/255.0, gamma) * 255
        """
        if kernel is None:
            kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        
        return cv2.filter2D(img, -1, kernel)
    
    def preprocess_image(self, img: np.ndarray, params: Dict) -> np.ndarray:
        """
        Complete image preprocessing pipeline
        
        Кроки:
        1. Bilateral filtering - шумо зниження зі збереженням країв
        2. Grayscale conversion
        3. Contrast enhancement
        """
        # Apply bilateral filter to smooth the image while preserving edges
        img_filtered = cv2.bilateralFilter(
            img, 
            params['bilateral_d'], 
            params['bilateral_sigma_color'], 
            params['bilateral_sigma_space']
        )
        
        # Convert to grayscale
        gray = cv2.cvtColor(img_filtered, cv2.COLOR_BGR2GRAY)
        
        # Enhance contrast
        contrast = self.contrast_enhancement(gray)
        
        return contrast
    
    def detect_edges(self, img: np.ndarray, params: Dict) -> np.ndarray:
        """
        Edge detection using Canny algorithm
        
        Альтернативи:
        - Sobel: cv2.Sobel()
        - Laplacian: cv2.Laplacian()
        - Scharr: cv2.Scharr()
        """
        canny = cv2.Canny(img, params['canny_T1'], params['canny_T2'])
        
        # Dilate edges to connect broken edge segments
        dilated = cv2.dilate(canny, self.dilation_kernel, iterations=params['dilate_iterations'])
        
        return dilated
    
    def detect_circles(self, img: np.ndarray, params: Dict) -> Optional[np.ndarray]:
        """
        Detect circles using Hough Circle Transform
        
        Альтернативи:
        - Template matching: cv2.matchTemplate()
        - Contour-based detection: cv2.findContours() + cv2.fitEllipse()
        - Deep learning approaches (YOLO, Faster R-CNN)
        """
        circles = cv2.HoughCircles(
            img, 
            cv2.HOUGH_GRADIENT, 
            dp=params['dp'],
            minDist=params['min_dist'],
            param1=params['param1'], 
            param2=params['param2'],
            minRadius=params['min_radius'], 
            maxRadius=params['max_radius']
        )
        
        return circles
    
    def circles_to_bbox(self, circles: np.ndarray, xywh: bool = True) -> List[List[float]]:
        """Convert circles to bounding boxes"""
        bboxes = []
        for circle in circles:
            if xywh:
                x_c, y_c, w, h = circle[0], circle[1], circle[2]*2, circle[2]*2
                bboxes.append([x_c, y_c, w, h])
            else:
                x1, y1, x2, y2 = (circle[0] - circle[2],
                                  circle[1] - circle[2], 
                                  circle[0] + circle[2],
                                  circle[1] + circle[2])
                bboxes.append([x1, y1, x2, y2])
        return bboxes
    
    def pixel_to_geo_coords(self, pixel_coords: Tuple[int, int], 
                           image_metadata: ImageMetadata) -> Tuple[float, float]:
        """
        Convert pixel coordinates to geographic coordinates
        
        Використовує Web Mercator проекцію (EPSG:3857)
        """
        x_pixel, y_pixel = pixel_coords
        width, height = image_metadata.image_size
        center_lat, center_lon = image_metadata.coordinates
        resolution = image_metadata.resolution
        
        # Calculate offset from image center in meters
        x_offset_pixels = x_pixel - width / 2
        y_offset_pixels = height / 2 - y_pixel  # Flip Y axis
        
        x_offset_meters = x_offset_pixels * resolution
        y_offset_meters = y_offset_pixels * resolution
        
        # Use geopy for accurate distance calculations
        from geopy import distance
        from geopy.distance import distance as geopy_distance
        
        # Calculate new coordinates
        start_point = (center_lat, center_lon)
        
        # Move east/west
        if x_offset_meters != 0:
            bearing_x = 90 if x_offset_meters > 0 else 270
            new_point = geopy_distance(meters=abs(x_offset_meters)).destination(
                start_point, bearing_x
            )
            center_lat, center_lon = new_point.latitude, new_point.longitude
        
        # Move north/south
        if y_offset_meters != 0:
            bearing_y = 0 if y_offset_meters > 0 else 180
            final_point = geopy_distance(meters=abs(y_offset_meters)).destination(
                (center_lat, center_lon), bearing_y
            )
            return final_point.latitude, final_point.longitude
        
        return center_lat, center_lon
    
    def pixels_to_meters(self, radius_pixels: int, resolution: float) -> float:
        """Convert pixel radius to meters"""
        return radius_pixels * resolution
    
    def detect_craters_single_image(self, image_name: str, 
                                  params: Dict = None) -> List[CraterDetection]:
        """Detect craters in a single image"""
        if params is None:
            params = self.default_params
        
        if image_name not in self.image_metadata:
            logger.error(f"No metadata found for image: {image_name}")
            return []
        
        metadata = self.image_metadata[image_name]
        img_path = os.path.join(self.input_folder, metadata.image_path)
        
        # Load image
        img = cv2.imread(img_path)
        if img is None:
            logger.error(f"Could not load image: {img_path}")
            return []
        
        # Preprocess image
        processed = self.preprocess_image(img, params)
        
        # Detect edges
        edges = self.detect_edges(processed, params)
        
        # Detect circles
        circles = self.detect_circles(edges, params)
        
        detections = []
        if circles is not None:
            circles = np.uint16(np.around(circles[0]))
            
            for circle in circles:
                x, y, r = circle
                
                # Convert to geographic coordinates
                geo_coords = self.pixel_to_geo_coords((x, y), metadata)
                radius_meters = self.pixels_to_meters(r, metadata.resolution)
                
                detection = CraterDetection(
                    pixel_coords=(x, y),
                    radius_pixels=r,
                    geo_coords=geo_coords,
                    radius_meters=radius_meters,
                    confidence=0.8  # Default confidence
                )
                detections.append(detection)
        
        return detections
    
    def detect_all_craters(self, params: Dict = None) -> Dict[str, List[CraterDetection]]:
        """Detect craters in all images"""
        if params is None:
            params = self.default_params
        
        results = {}
        for image_name in self.image_metadata.keys():
            logger.info(f"Processing image: {image_name}")
            detections = self.detect_craters_single_image(image_name, params)
            results[image_name] = detections
            logger.info(f"Found {len(detections)} craters in {image_name}")
        
        self.detection_results = results
        return results
    
    def save_results(self, filename: str = "crater_detections.json"):
        """Save detection results to JSON file"""
        output_path = os.path.join(self.output_folder, filename)
        
        # Convert results to serializable format
        serializable_results = {}
        for image_name, detections in self.detection_results.items():
            serializable_results[image_name] = []
            for detection in detections:
                serializable_results[image_name].append({
                    'pixel_coords': detection.pixel_coords,
                    'radius_pixels': detection.radius_pixels,
                    'geo_coords': detection.geo_coords,
                    'radius_meters': detection.radius_meters,
                    'confidence': detection.confidence
                })
        
        with open(output_path, 'w') as f:
            json.dump(serializable_results, f, indent=2)
        
        logger.info(f"Results saved to: {output_path}")
    
    def save_metadata(self, filename: str = "detection_metadata.json"):
        """Save metadata about detection process"""
        output_path = os.path.join(self.output_folder, filename)
        
        metadata = {
            'input_folder': self.input_folder,
            'output_folder': self.output_folder,
            'total_images': len(self.image_metadata),
            'total_detections': sum(len(detections) for detections in self.detection_results.values()),
            'parameters_used': self.optimized_params if self.optimized_params else self.default_params,
            'images_metadata': {}
        }
        
        for name, img_meta in self.image_metadata.items():
            metadata['images_metadata'][name] = {
                'coordinates': img_meta.coordinates,
                'resolution': img_meta.resolution,
                'zoom_level': img_meta.zoom_level,
                'image_size': img_meta.image_size,
                'detections_count': len(self.detection_results.get(name, []))
            }
        
        with open(output_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        logger.info(f"Metadata saved to: {output_path}")
    
    def visualize_detections(self, image_name: str, save_image: bool = True):
        """Visualize detected craters on image"""
        if image_name not in self.image_metadata:
            logger.error(f"No metadata found for image: {image_name}")
            return
        
        metadata = self.image_metadata[image_name]
        img_path = os.path.join(self.input_folder, metadata.image_path)
        
        img = cv2.imread(img_path)
        if img is None:
            logger.error(f"Could not load image: {img_path}")
            return
        
        # Draw detections
        detections = self.detection_results.get(image_name, [])
        vis_img = img.copy()
        
        for detection in detections:
            x, y = detection.pixel_coords
            r = detection.radius_pixels
            
            # Draw circle
            cv2.circle(vis_img, (x, y), r, (0, 255, 0), 2)
            # Draw center
            cv2.circle(vis_img, (x, y), 2, (0, 0, 255), 3)
            # Add text with radius
            cv2.putText(vis_img, f'r={detection.radius_meters:.1f}m', 
                       (x-20, y-r-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        if save_image:
            output_path = os.path.join(self.output_folder, f"{image_name}_detections.jpg")
            cv2.imwrite(output_path, vis_img)
            logger.info(f"Visualization saved to: {output_path}")
        
        return vis_img

In [ ]:


class OptunaOptimizer:
    """Optuna-based parameter optimization for crater detection"""
    
    def __init__(self, processor: SatelliteImageProcessor, ground_truth_dir: str = None):
        self.processor = processor
        self.ground_truth_dir = ground_truth_dir
    
    def calculate_iou(self, boxA: List[float], boxB: List[float]) -> float:
        """Calculate Intersection over Union (IoU) for two bounding boxes"""
        x1A, y1A, x2A, y2A = boxA
        x1B, y1B, x2B, y2B = boxB
        
        x1_intersection = max(x1A, x1B)
        y1_intersection = max(y1A, y1B)
        x2_intersection = min(x2A, x2B)
        y2_intersection = min(y2A, y2B)
        
        intersection_area = max(0, x2_intersection - x1_intersection) * \
                          max(0, y2_intersection - y1_intersection)
        
        boxA_area = (x2A - x1A) * (y2A - y1A)
        boxB_area = (x2B - x1B) * (y2B - y1B)
        union_area = boxA_area + boxB_area - intersection_area
        
        return intersection_area / union_area if union_area > 0 else 0
    
    def calculate_metrics(self, predictions: List[List[float]], 
                         ground_truths: List[List[float]], 
                         iou_threshold: float = 0.5) -> Metrics:
        """Calculate precision, recall, and F1 score"""
        TP = 0
        FP = 0
        FN = 0
        detected = []
        
        for pred in predictions:
            found_match = False
            for truth in ground_truths:
                iou = self.calculate_iou(pred, truth)
                if iou >= iou_threshold and truth not in detected:
                    TP += 1
                    detected.append(truth)
                    found_match = True
                    break
            if not found_match:
                FP += 1
        
        FN = len(ground_truths) - len(detected)
        
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        return Metrics(
            TP=TP, FP=FP, FN=FN,
            precision=precision, recall=recall, f1_score=f1_score
        )
    
    def objective(self, trial):
        """Optuna objective function"""
        # Define parameter search space
        params = {
            'canny_T1': trial.suggest_int('canny_T1', 50, 150),
            'canny_T2': trial.suggest_int('canny_T2', 20, 100),
            'dp': trial.suggest_float('dp', 1.0, 2.0),
            'min_dist': trial.suggest_int('min_dist', 20, 100),
            'param1': trial.suggest_int('param1', 30, 100),
            'param2': trial.suggest_int('param2', 10, 50),
            'min_radius': trial.suggest_int('min_radius', 3, 20),
            'max_radius': trial.suggest_int('max_radius', 15, 100),
            'bilateral_d': trial.suggest_int('bilateral_d', 5, 15),
            'bilateral_sigma_color': trial.suggest_int('bilateral_sigma_color', 50, 150),
            'bilateral_sigma_space': trial.suggest_int('bilateral_sigma_space', 50, 150),
            'dilate_iterations': trial.suggest_int('dilate_iterations', 1, 5)
        }
        
        # Run detection with current parameters
        results = self.processor.detect_all_craters(params)
        
        if not self.ground_truth_dir:
            # If no ground truth, use detection count as proxy metric
            total_detections = sum(len(detections) for detections in results.values())
            return total_detections
        
        # Calculate metrics against ground truth
        total_metrics = Metrics(TP=0, FP=0, FN=0, precision=0, recall=0, f1_score=0)
        
        for image_name, detections in results.items():
            # Convert detections to bounding boxes
            pred_bboxes = []
            for det in detections:
                x, y, r = det.pixel_coords[0], det.pixel_coords[1], det.radius_pixels
                pred_bboxes.append([x-r, y-r, x+r, y+r])
            
            # Load ground truth if available
            gt_path = os.path.join(self.ground_truth_dir, f"{image_name}.txt")
            if os.path.exists(gt_path):
                with open(gt_path, 'r') as f:
                    gt_lines = f.readlines()
                
                gt_bboxes = []
                for line in gt_lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        # Assuming YOLO format: class x_center y_center width height
                        _, x_c, y_c, w, h = map(float, parts)
                        img_w, img_h = self.processor.image_metadata[image_name].image_size
                        x_c *= img_w
                        y_c *= img_h
                        w *= img_w
                        h *= img_h
                        gt_bboxes.append([x_c - w/2, y_c - h/2, x_c + w/2, y_c + h/2])
                
                metrics = self.calculate_metrics(pred_bboxes, gt_bboxes)
                total_metrics.TP += metrics.TP
                total_metrics.FP += metrics.FP
                total_metrics.FN += metrics.FN
        
        # Calculate overall metrics
        total_metrics.precision = total_metrics.TP / (total_metrics.TP + total_metrics.FP) \
                                 if (total_metrics.TP + total_metrics.FP) > 0 else 0
        total_metrics.recall = total_metrics.TP / (total_metrics.TP + total_metrics.FN) \
                              if (total_metrics.TP + total_metrics.FN) > 0 else 0
        total_metrics.f1_score = 2 * total_metrics.precision * total_metrics.recall / \
                                (total_metrics.precision + total_metrics.recall) \
                                if (total_metrics.precision + total_metrics.recall) > 0 else 0
        
        return total_metrics.f1_score
    
    def optimize(self, n_trials: int = 100) -> Dict:
        """Run optimization"""
        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=n_trials)
        
        logger.info("Optimization completed!")
        logger.info(f"Best parameters: {study.best_params}")
        logger.info(f"Best value: {study.best_value}")
        
        # Update processor with optimized parameters
        optimized_params = self.processor.default_params.copy()
        optimized_params.update(study.best_params)
        self.processor.optimized_params = optimized_params
        
        return study.best_params

In [ ]:
def main():
    """Main function to demonstrate usage"""
    # Initialize processor
    input_folder = "satellite_images"  # Folder with images from SatelliteImageRetriever
    processor = SatelliteImageProcessor(input_folder)
    
    # Optional: Optimize parameters
    optimizer = OptunaOptimizer(processor)
    # best_params = optimizer.optimize(n_trials=50)
    
    # Detect craters
    results = processor.detect_all_craters()
    
    # Save results
    processor.save_results()
    processor.save_metadata()
    
    # Visualize results for each image
    for image_name in processor.image_metadata.keys():
        processor.visualize_detections(image_name)
    
    logger.info("Crater detection completed!")

if __name__ == "__main__":
    main()